In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from torchvision.utils import save_image

In [ ]:
if (os.path.exists("./data")) == False:
    os.mkdir("data")

In [ ]:
dir_pokemon = "./archive/pokemon"
files_pokemon = os.listdir(dir_pokemon + '/pokemon')

In [ ]:
transform=transforms.Compose([
                               transforms.Resize(256),
                               transforms.CenterCrop(256),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])
batch_size = 400

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


dataset = ImageFolder(root = dir_pokemon, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last = True)

In [ ]:
first_batch = next(iter(dataloader))
plt.figure(figsize=(6, 6))
plt.axis("off")
plt.title("Batch of training images")
plt.imshow(vutils.make_grid(first_batch[0][:64], padding=2, normalize=True).cpu().numpy().transpose(1,2,0))
plt.show()

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(128, 512, kernel_size=(4, 4), stride=(1, 1), bias=False),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=0, bias=False),
            nn.AdaptiveAvgPool2d(1),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1)

# Train

In [ ]:
criterion = nn.BCELoss()

generator = Generator()
discriminator = Discriminator()


generator.to(device)
discriminator.to(device)

optimizerD = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 1000

In [ ]:
# Training Loop
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        image, image_label = data[0].to(device), data[1].to(device)

        # Train discriminator
        discriminator.zero_grad()

        label = torch.full((batch_size,), 1, dtype=torch.float, device=device)

        output = discriminator(image).view(-1)

        loss_discriminator_real = criterion(output, label)
        loss_discriminator_real.backward()

        noise = torch.randn(batch_size, 128, 1, 1, device=device)
        fake_image = generator(noise)
        label.fill_(0)

        output = discriminator(fake_image.detach()).view(-1)

        loss_discriminator_fake = criterion(output, label)
        loss_discriminator_fake.backward()

        errD = loss_discriminator_real + loss_discriminator_fake
        optimizerD.step()

        # Train generator
        generator.zero_grad()
        label.fill_(1)
        output = discriminator(fake_image).view(-1)

        loss_generator = criterion(output, label)
        loss_generator.backward()
        optimizerG.step()

    if (epoch % 10 == 0):
        with torch.no_grad():
            fixed_noise = torch.randn(64, 128, 1, 1, device=device)
            fake_images = generator(fixed_noise).detach().cpu()
            loss_on_real = loss_discriminator_real.detach().cpu()
            loss_on_fake = loss_discriminator_fake.detach().cpu()

        print(f'epoch: {epoch}, loss on real: {loss_on_real}, loss on fake: {loss_on_fake}')

        save_image(fake_images, f'./data/fake_images_epoch_{epoch:04d}.png', nrow=8, normalize=True)